In [10]:
# Import
import sys
import numpy as np
import pandas as pd
from keras.src.callbacks import EarlyStopping
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras

In [11]:
# Set random seeds
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# Combine datasets

In [19]:
# Read the datasets
real_images = pd.read_csv("real_images.csv")
fake_images = pd.read_csv("almost_real_fake_images.csv")

In [21]:
# Drop some real imagesso that there are as many real images as fake images
real_images.drop(real_images.tail(15_000).index,inplace=True)
print(real_images.shape, fake_images.shape)

(10000, 784) (10000, 785)


In [22]:
# Normalize values between 0 and 1
real_images /= 255.
fake_images /= 255.

In [23]:
# Add label column
real_images["label"] = 1.0
fake_images["label"] = 0.0

In [24]:
# Combine the two datasets
images = pd.concat([real_images, fake_images], ignore_index=True)

In [25]:
# Shuffle
images = images.sample(frac=1).reset_index(drop=True)

In [26]:
# Split into training and test sets
features = images.drop("label", axis=1)
labels = images["label"]

# Create discriminator models

In [29]:
if sys.platform.startswith('darwin'):
    optimizer = tf.keras.optimizers.legacy.SGD
else:
    optimizer = "sgd"
    
TOTAL_MODELS = 10
TRAINING_SIZE = 16_000
TEST_SIZE = 4_000
MODEL_TRAINING_SIZE = int(TRAINING_SIZE / TOTAL_MODELS)
MODEL_TEST_SIZE = int(TEST_SIZE / TOTAL_MODELS)

for i in range(TOTAL_MODELS):
    # create training and test set for the model
    start_idx = i * MODEL_TRAINING_SIZE
    end_idx = (i + 1) * MODEL_TRAINING_SIZE

    m_x_test = features[end_idx:end_idx + MODEL_TEST_SIZE]
    m_x_train = features[start_idx:end_idx]

    m_y_test = labels[end_idx:end_idx + MODEL_TEST_SIZE]
    m_y_train = labels[start_idx:end_idx]
    # Create model
    model = keras.models.Sequential([
        keras.layers.Input(shape=[784]),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid")
    ])
    # Compile model
    model.compile(loss="binary_crossentropy",
                  optimizer="sgd",
                  metrics=["accuracy"])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    # Train model
    model.fit(m_x_train, m_y_train, epochs=5,
              validation_data=(m_x_test, m_y_test))
    # Save model
    model.save(f"discriminator_models/discriminator_{i+1}.keras")

Epoch 1/5
50/50 [==============================] - 1s 7ms/step - loss: 0.5269 - accuracy: 0.9106 - val_loss: 0.4088 - val_accuracy: 1.0000
Epoch 2/5
50/50 [==============================] - 0s 3ms/step - loss: 0.3878 - accuracy: 1.0000 - val_loss: 0.3428 - val_accuracy: 1.0000
Epoch 3/5
50/50 [==============================] - 0s 3ms/step - loss: 0.3380 - accuracy: 1.0000 - val_loss: 0.3046 - val_accuracy: 1.0000
Epoch 4/5
50/50 [==============================] - 0s 3ms/step - loss: 0.3039 - accuracy: 1.0000 - val_loss: 0.2755 - val_accuracy: 1.0000
Epoch 5/5
50/50 [==============================] - 0s 3ms/step - loss: 0.2765 - accuracy: 1.0000 - val_loss: 0.2515 - val_accuracy: 1.0000
Epoch 1/5
50/50 [==============================] - 1s 7ms/step - loss: 0.5153 - accuracy: 0.9588 - val_loss: 0.4221 - val_accuracy: 1.0000
Epoch 2/5
50/50 [==============================] - 0s 3ms/step - loss: 0.3801 - accuracy: 1.0000 - val_loss: 0.3494 - val_accuracy: 1.0000
Epoch 3/5
50/50 [==========